<a href="https://colab.research.google.com/github/zealandia17/Predicting-hurricane/blob/main/LV_856_113_20W_2SH_P_Barbosa_Ferreira_Nurul_Fatma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Machine Learning Final Assigment**

---



---


Vitória Barbosa Ferreira | *12031088*

Zealandia Sarah Nurul Fatma | *12031257*

This assigment consists in a hands on machine learning methods taught on the discipline Introduction to DataScience and Machine Learning for Geospatial Data. ***It aims to predict the status of a hurricane based on its location and maximum wind.*** The workflow of the analysis is divided into fours steps: Data Inspection, Data Handling, Data Visualization and Hurricane Status Prediction.
The Hurricane Status Prediction is performed using three methods:

> a. Decision Trees 

> b. KNN 

> c. Naive Bayes 













##**Data Inspection**

In [ ]:
from google.colab import drive, files
drive.mount('/content/gdrive')

%matplotlib inline
import pandas as pd
import numpy as np
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import re
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import  accuracy_score, average_precision_score
from sklearn.externals import joblib
from sklearn import tree
from graphviz import Source
import pydot
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import  GaussianNB
import plotly.graph_objects as go
import plotly.express as px

Mounted at /content/gdrive


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
df = pd.read_csv('gdrive/My Drive/Colab Notebooks/atlantic.csv', na_values=[-999,-99])
#df = pd.read_csv('gdrive/My Drive/Colab Notebooks/atlantic.csv', sep=';', na_values=[-999,-99])
df.head(3)


,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,Low Wind NE,Low Wind SE,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
0,AL011851,UNNAMED,18510625,0,,HU,28.0N,94.8W,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AL011851,UNNAMED,18510625,600,,HU,28.0N,95.4W,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AL011851,UNNAMED,18510625,1200,,HU,28.0N,96.0W,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.shape

(49105, 22)

In [ ]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49105 entries, 0 to 49104
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                49105 non-null  object 
 1   Name              49105 non-null  object 
 2   Date              49105 non-null  int64  
 3   Time              49105 non-null  int64  
 4   Event             49105 non-null  object 
 5   Status            49105 non-null  object 
 6   Latitude          49105 non-null  object 
 7   Longitude         49105 non-null  object 
 8   Maximum Wind      48767 non-null  float64
 9   Minimum Pressure  18436 non-null  float64
 10  Low Wind NE       5921 non-null   float64
 11  Low Wind SE       5921 non-null   float64
 12  Low Wind SW       5921 non-null   float64
 13  Low Wind NW       5921 non-null   float64
 14  Moderate Wind NE  5921 non-null   float64
 15  Moderate Wind SE  5921 non-null   float64
 16  Moderate Wind SW  5921 non-null   float6

,Date,Time,Maximum Wind,Minimum Pressure,Low Wind NE,Low Wind SE,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
count,4.910500e+04,49105.000000,48767.000000,18436.000000,5921.000000,5921.000000,5921.000000,5921.000000,5921.000000,5921.000000,5921.000000,5921.000000,5921.000000,5921.000000,5921.000000,5921.000000
mean,1.949802e+07,910.125975,53.051695,992.244250,81.865394,76.518325,48.647188,59.156393,24.641952,23.029894,15.427293,18.403141,8.110117,7.357710,5.130890,6.269211
std,4.461850e+05,671.043363,24.747884,19.113748,88.097930,87.563153,75.209183,77.568911,41.592337,42.017821,32.105372,35.411258,19.792002,18.730334,14.033464,16.876623
min,1.851062e+07,0.000000,10.000000,882.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.911110e+07,600.000000,35.000000,984.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.956093e+07,1200.000000,50.000000,999.000000,60.000000,60.000000,0.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.989081e+07,1800.000000,70.000000,1006.000000,130.000000,120.000000,75.000000,90.000000,40.000000,35.000000,20.000000,30.000000,0.000000,0.000000,0.000000,0.000000
max,2.015111e+07,2330.000000,165.000000,1024.000000,710.000000,600.000000,640.000000,530.000000,360.000000,300.000000,330.000000,360.000000,180.000000,250.000000,150.000000,180.000000


##**Data Handling**

Data handling or cleaning is one of the most import steps in any machine learning project, since it aims to minimize the impact that non-handled data may cause when running a machine learning algorithm. In this regard, the main steps of this section consisted in dropping NaN values in the columns used to run the model; checking for duplicates and filtering by year. In addition, other few data manipulation had to be done, such as setting the date column to datetime in order to extract the year (visualization purpose); dropping 'NSWE' from the coordinates and multipling the longitude columns by -1 (for the models and for visualizing the hurricanes distribution). 

In [ ]:
df.columns = df.columns.str.replace(' ','_',regex=True)
df.columns = [col.lower() for col in df]
df.head(2)

,id,name,date,time,event,status,latitude,longitude,maximum_wind,minimum_pressure,low_wind_ne,low_wind_se,low_wind_sw,low_wind_nw,moderate_wind_ne,moderate_wind_se,moderate_wind_sw,moderate_wind_nw,high_wind_ne,high_wind_se,high_wind_sw,high_wind_nw
0,AL011851,UNNAMED,18510625,0,,HU,28.0N,94.8W,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AL011851,UNNAMED,18510625,600,,HU,28.0N,95.4W,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Handling datetime formats
df['time'] = df['time'].apply(lambda x: "%04d" % x) # time from int to string
df['date_time'] = df['date'].apply(lambda x: str(x)) + df['time'] # new column merging time and date
df['date_time'] = df['date_time'].apply(lambda x: datetime.datetime.strptime(x, '%Y%m%d%H%M')) # from string to datetime.datetime
df['year']=df['date_time'].dt.year


In [ ]:
#filtering by date 1950-2015 (have less NaN values in the target columns)
booleans = []

for date in df['date']:
  if date > 19500000:
    booleans.append(True)
  else:
    booleans.append(False)
  
  date_range = pd.Series(booleans)

new_df = df[date_range]
new_df.head()


,id,name,date,time,event,status,latitude,longitude,maximum_wind,minimum_pressure,low_wind_ne,low_wind_se,low_wind_sw,low_wind_nw,moderate_wind_ne,moderate_wind_se,moderate_wind_sw,moderate_wind_nw,high_wind_ne,high_wind_se,high_wind_sw,high_wind_nw,date_time,year
21879,AL011950,ABLE,19500812,0000,,TS,17.1N,55.5W,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1950-08-12 00:00:00,1950
21880,AL011950,ABLE,19500812,0600,,TS,17.7N,56.3W,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1950-08-12 06:00:00,1950
21881,AL011950,ABLE,19500812,1200,,TS,18.2N,57.4W,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1950-08-12 12:00:00,1950
21882,AL011950,ABLE,19500812,1800,,TS,19.0N,58.6W,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1950-08-12 18:00:00,1950
21883,AL011950,ABLE,19500813,0000,,TS,20.0N,60.0W,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1950-08-13 00:00:00,1950


In [ ]:
new_df.dtypes

id                          object
name                        object
date                         int64
time                        object
event                       object
status                      object
latitude                    object
longitude                   object
maximum_wind               float64
minimum_pressure           float64
low_wind_ne                float64
low_wind_se                float64
low_wind_sw                float64
low_wind_nw                float64
moderate_wind_ne           float64
moderate_wind_se           float64
moderate_wind_sw           float64
moderate_wind_nw           float64
high_wind_ne               float64
high_wind_se               float64
high_wind_sw               float64
high_wind_nw               float64
date_time           datetime64[ns]
year                         int64
dtype: object

In [ ]:
#handling coordinate formats

new_df['latitude'] = new_df['latitude'].astype(str)
new_df['longitude'] = new_df['longitude'].astype(str)

new_df['longitude'] = new_df['longitude'].map(lambda x: x.rstrip('W'))
new_df['latitude'] = new_df ['latitude'].map(lambda x: x.rstrip ('N'))
new_df['longitude'] = new_df['longitude'].map(lambda x: x.rstrip('E'))
new_df['latitude'] = new_df ['latitude'].map(lambda x: x.rstrip ('S'))

#converting coordinate format to float
new_df['latitude'] = new_df['latitude'].astype(float)
new_df['longitude'] = new_df['longitude'].astype(float)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [ ]:
#convert longitude to negative
new_df['longitude'] = (new_df['longitude'] * -1)
new_df.head(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,name,date,time,event,status,latitude,longitude,maximum_wind,minimum_pressure,low_wind_ne,low_wind_se,low_wind_sw,low_wind_nw,moderate_wind_ne,moderate_wind_se,moderate_wind_sw,moderate_wind_nw,high_wind_ne,high_wind_se,high_wind_sw,high_wind_nw,date_time,year
21879,AL011950,ABLE,19500812,0000,,TS,17.1,-55.5,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1950-08-12 00:00:00,1950
21880,AL011950,ABLE,19500812,0600,,TS,17.7,-56.3,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1950-08-12 06:00:00,1950


In [ ]:
#droping unecessary columns
new_df = new_df.drop(['id','name','event','date', 'time'], axis=1)

KeyError: ignored

In [ ]:
#handling NaN values
new_df = new_df.dropna(subset=['status', 'maximum_wind','latitude','longitude'])
new_df.head(2)

,status,latitude,longitude,maximum_wind,minimum_pressure,low_wind_ne,low_wind_se,low_wind_sw,low_wind_nw,moderate_wind_ne,moderate_wind_se,moderate_wind_sw,moderate_wind_nw,high_wind_ne,high_wind_se,high_wind_sw,high_wind_nw,date_time,year
21879,TS,17.1,-55.5,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1950-08-12 00:00:00,1950
21880,TS,17.7,-56.3,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1950-08-12 06:00:00,1950


In [ ]:
#checking for duplicates (same status, maximum wind and lat/long)
new_df.duplicated(subset=['status', 'maximum_wind','latitude','longitude'], keep='first')

21879    False
21880    False
21881    False
21882    False
21883    False
         ...  
49100    False
49101    False
49102    False
49103    False
49104    False
Length: 26888, dtype: bool

In [ ]:
#new dataframe with the number of hurricanes per year

count = new_df['year'].value_counts()
count_df = pd.DataFrame(count).reset_index()
count_df = count_df.rename(columns={"index": "year", "year": "count"}).sort_values(by = "year", ascending = True)
count_df

,year,count
12,1950,540
29,1951,415
48,1952,310
34,1953,391
41,1954,360
...,...,...
8,2011,557
4,2012,654
43,2013,346
50,2014,300


In [ ]:
#new dataframe with the count of status
count_status = df['status'].value_counts()
status_df = pd.DataFrame(count_status).reset_index()
status_df = status_df.rename(columns={"index": "status", "status": "count"}).sort_values(by = "count", ascending = False)
status_df

,status,count
0,TS,17804
1,HU,14537
2,TD,9891
3,EX,4798
4,LO,1005
5,SS,557
6,SD,293
7,WV,116
8,DB,104


In [ ]:
count_df.max()

year     2015
count     907
dtype: int64

##**Data Visualization**

This section provides simple visualizations about the data. The first graph shows the frequency of each status class, being TD (Tropical cyclone of tropical depression intensity) the most frequent type. The second graph illustrates the distribution of the atlantic hurricanes. It is possible to infer that the location plays an important role in the wind speed, therefore, also in the hurricane status. 


**Status legend:**
* TD – Tropical cyclone of tropical depression intensity (< 34 knots)
* TS – Tropical cyclone of tropical storm intensity (34-63 knots)
* HU – Tropical cyclone of hurricane intensity (> 64 knots)
* EX – Extratropical cyclone (of any intensity)
* SD – Subtropical cyclone of subtropical depression intensity (< 34 knots)
* SS – Subtropical cyclone of subtropical storm intensity (> 34 knots)
* LO – A low that is neither a tropical cyclone, a subtropical cyclone, nor an extratropical cyclone (of any intensity)
* WV – Tropical Wave (of any intensity)
* DB – Disturbance (of any intensity)

In [ ]:
#number of hurricanes per year

fig = px.bar(count_df, x='year', y='count', 
             title='Number of hurricanes per year (1950 - 2015)',
             labels={'count': 'number of hurricanes\n'},
             color_discrete_sequence=['purple']
             )

fig.show()

In [ ]:
#bar chart for status occurances

fig = px.bar(status_df, x='status', y= 'count', 
             title='Frequency of Status (1950 - 2015)',
             color='count')

fig.show()


In [ ]:
status_df.columns

Index(['status', 'count'], dtype='object')

In [ ]:
fig4 = go.Figure(go.Densitymapbox(
                                  lat=new_df.latitude, 
                                  lon=new_df.longitude,
                                  z=new_df['maximum_wind'],
                                  radius=30, 
                                  opacity =0.5,
                                  hoverinfo = 'none',
                                 ))
fig4.update_layout( title = "Heatmap of Maximum Wind Speed (knots)", 
                   title_x = 0.5, 
                   mapbox_style= "stamen-terrain", 
                   mapbox_center_lon= -76, 
                   mapbox_center_lat = 17)

fig4.show()

##**Huricane Status Prediction**


---












In [ ]:
new_df.columns

Index(['status', 'latitude', 'longitude', 'maximum_wind', 'minimum_pressure',
       'low_wind_ne', 'low_wind_se', 'low_wind_sw', 'low_wind_nw',
       'moderate_wind_ne', 'moderate_wind_se', 'moderate_wind_sw',
       'moderate_wind_nw', 'high_wind_ne', 'high_wind_se', 'high_wind_sw',
       'high_wind_nw', 'date_time', 'year'],
      dtype='object')

**1.   Decision Tree**

Decision Tree is one of Supervised Classification method which using Decision Node as Root Node to create any decision and have multiple branches creates Leaf Nodes as the output of those decisions.

Based on prediction that we've made; predict the next Status of Hurricane, we try Decision Tree as first Machine Learning technique using **Maximum Wind** and **Location Coordinate** as Decision Nodes to create the Status Hurricane prediction. Following as below steps:

1. Defining the training and testing dataset
2. Setting the model and prediction based on training data
3. Running the prediction model
4. Evaluating the model performance accuracy

In [ ]:
#Defining the training and testing dataset
x = new_df.drop(['status','minimum_pressure',
       'low_wind_ne', 'low_wind_se', 'low_wind_sw', 'low_wind_nw',
       'moderate_wind_ne', 'moderate_wind_se', 'moderate_wind_sw',
       'moderate_wind_nw', 'high_wind_ne', 'high_wind_se', 'high_wind_sw',
       'high_wind_nw', 'date_time', 'year'], axis = 1) 
#Dropping irrelevant columns, (only use; 'maximum_wind, 'lat', 'long' as input parameters X)

Y = new_df['status']
#Select column 'status' as target variable y

x_train, x_test, Y_train, Y_test = train_test_split(x, Y, test_size = 0.2, random_state=42)
#Splitting X and y into training data and test data within proportion of 80% as training and 20% as test

#Setting the model and the prediction based on train samples
we = DecisionTreeClassifier(random_state=42)
#Create classifier object
we = we.fit(x_train,Y_train)
#Train the classifier using the training data

#Running the prediction model
predictions = we.predict(x_test)
predictions


#Calculates the score accuracy
scoree = accuracy_score(Y_test,predictions)
scoree

predictions_cond_dt = we.predict([[30.3,55.6,25]])
predictions_cond_dt

array([' LO'], dtype=object)

In [ ]:
# decision tree 2
new_df = new_df.dropna(subset=['status', 'maximum_wind','latitude','longitude', 'low_wind_ne', 'low_wind_se', 'low_wind_nw', 'moderate_wind_se', 'moderate_wind_nw','high_wind_ne','high_wind_se','high_wind_sw','high_wind_nw'])
X = new_df.drop(['status', 'date_time'], axis = 1)

y = new_df['status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)

model = DecisionTreeClassifier(random_state=1)
model.fit(X_train, y_train)

prediction = model.predict(X_test)

score = accuracy_score(y_test, prediction)
score



0.8793248945147679

In [ ]:
#Predicting one specific scenario
predictions_cond_dt = models.predict([[30.3,55.6,981.0, 0.0, 50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,2015]])
predictions_cond_dt

array([' HU'], dtype=object)

In [ ]:
#Export the decision tree model
# tree.export_graphviz(model, out_file='hurricane_status.dot',
#                      feature_names=['latitude', 'longitude', 'maximum_wind', 'latitude_hemisphere','longitude_hemisphere'], 
#        class_names = sorted(y.unique()),
#        label = 'all',
#        rounded = True,
#        filled = True) 


In [ ]:
#Downloading the decision tree model 

# Source.from_file('hurricane_status.dot', format='png') #for visualising the model on the google colab (too big)

# (graph,) = pydot.graph_from_dot_file('hurricane_status.dot')
# graph.write_pdf('hurricane_status.pdf')

# files.download('hurricane_status.pdf') #for downloading the model in pdf format


**Prediction Result**


Decision Tree employs *maximum_wind*, *latitude*, and *longitude* columns  as root trees, the machine will learn to partition on certain values from those recursively resulting leaf nodes for each decision node. In leaf nodes show the class of status prediction. 
Prediction result for X_test stored in arrays. Evaluting the prediction could be calculate by comparing the actual test data and the prediction model.Accuracy score for this method is around **82%** considered as good accuracy. The specific prediction also could be created by identify certain values of *maximum_wind*, *latitude*, and *longitude*.

Decision trees technique is easy to understand and interpret by flowchart of decision nodes, user could easily following the logic behind. Other pros is without normalizing columns data still can be used. Otherwise decision tree is method that sensitive of noisy data, and a glance of variation could result differentially. 



**2.   KNN**

KNN stands for K-nearest neighbors algorithm, consisting of a neighbouring based machine learning method. An object is classified with the most common label among its k nearest neighbours. For instace if k = 3 , the target object is assigned to the most frequent class among the 3 closest neighbours (considering euclidian distance). 

The same independent variables were used to predict the hurricane status with KNN. Moreover, under the sklearn package, the workflow is similar to the Decision Tree algorithm, which also consisted in:

1. Defining the training and testing dataset
2. Setting the model and prediction based on training data
3. Running the prediction model
4. Evaluating the model performance accuracy

The value of k were tested under a set o odd numbers, even considering k=sqrt(n). In this regard, K = 15 showed the best performance in terms of accuracy. The euclidian distance is the metrics used to measure the distance.




In [ ]:
#Defyning the training and testing dataset

X_knn = X 

y_knn = y

X_knn_train, X_knn_test, y_knn_train, y_knn_test = train_test_split(X_knn,y_knn, test_size = 0.2)

#Feature scaling
sc_X = StandardScaler()
X_knn_train = sc_X.fit_transform(X_knn_train)
X_knn_test = sc_X.transform(X_knn_test)

#Fitting the classifier
classifier = KNeighborsClassifier(n_neighbors=15, p=2, metric= 'euclidean', weights='uniform') 
classifier.fit(X_knn_train, y_knn_train)

#Prediction of the test set results
y_pred = classifier.predict(X_knn_test)
y_pred

#Accuracy score
score = accuracy_score(y_knn_test,y_pred)
score

0.8709557456303458

In [ ]:
#Predicting one specific scenario
predictions_cond_knn = model.predict([[17.5,55.6,25]])
predictions_cond_knn

array([' TD'], dtype='<U3')

**Prediction Result**


KNN performed slightly better than the Decision Tree algorithm with an accuracy score around **87%**. The main advantages of KNN is that the algorithm is simple to understand and it performs well with multiclass tasks, such as predicting the hurricane status. Among the setbacks stands out choosing an optimal value for k, which required multiple tests. Moreover, as it can be seen from the visualization section, the database is not very well balance, which may lead to a lot of more preference to the most frequent labels TS and HU in the voting process. 

**3.   Naive Bayes**


Naive Bayes using *Bayes hypothesis probability theorem* as principle combined with assumption the occurrence between features is independent. There are several types of Naive Bayes, in this case we use Gaussian Naive Bayes method which assuming a Gaussian distribution for training data by calculating mean and standard deviation for each class of Hurricane status. Therefore, mean and standard deviation are addition values for each class probability as well as input variable per class. 



In [ ]:
#Defyning the training and testing dataset

input = X  

target = y

X_nb_train, X_nb_test, y_nb_train, y_nb_test = train_test_split(input,target, test_size = 0.2)

#Fitting the classifier

model = GaussianNB()
model.fit(X_nb_train,y_nb_train)

predictions = model.predict(X_nb_test)
predictions

# model.score(X_nb_test,y_nb_test)

score = model.score(X_nb_test,y_nb_test)
score

0.8566381554481219

In [ ]:
#Predicting one specific scenario
predictions_cond_nb = model.predict([[17.5,55.6,25]])
predictions_cond_nb

array([' TD'], dtype='<U3')

In [ ]:
model.predict_proba(X_nb_test[:10]) #probability of 10 rows samples for each class in the model

array([[4.96334028e-18, 1.40398251e-01, 8.10575249e-01, 1.41275717e-11,
        1.78383690e-22, 1.40746104e-02, 1.36541072e-31, 3.49518891e-02,
        4.68077961e-15],
       [2.05073556e-27, 7.25935977e-04, 9.99271693e-01, 4.96254010e-23,
        1.64207269e-46, 6.81450832e-07, 3.95308457e-67, 1.68991465e-06,
        5.01757360e-24],
       [3.43476179e-02, 1.07690547e-03, 8.29820432e-04, 1.70369876e-01,
        3.08731267e-03, 2.76984122e-05, 7.69603760e-01, 1.08182052e-02,
        9.83880427e-03],
       [1.45617145e-02, 3.40939269e-03, 3.57558735e-03, 5.49541157e-02,
        2.39581298e-02, 2.87288431e-03, 7.96843080e-01, 9.34256169e-02,
        6.39947904e-03],
       [2.21426863e-23, 1.59831937e-02, 9.83407932e-01, 6.89284676e-17,
        1.50916599e-33, 1.97454429e-04, 7.23485897e-48, 4.11420297e-04,
        1.27097427e-19],
       [1.83210303e-05, 5.90311336e-04, 1.88148697e-04, 6.65757398e-02,
        3.83081545e-06, 1.43233875e-06, 9.21285754e-01, 1.11144924e-02,
        2.2

In [ ]:
#Checking the std
X_nb_train['maximum_wind'].describe() 

count    21510.000000
mean        48.773222
std         25.110803
min         10.000000
25%         30.000000
50%         40.000000
75%         65.000000
max        160.000000
Name: maximum_wind, dtype: float64

**Prediction Result**

In this model, both input (X) and target (y) splitted to test and train data within proportion of test 80% and train 20%. Accuracy score shows around **85.6%** determined as good accuracy.

Benefit from using Naive Bayes is while computing the model this technique performed relative faster, low computation cost, and works in discrete variable. Otherwise, the assumption of independent features in practice is hard to find, even though still works pretty well.  

**Final considerations**

This assigment tested three supervised machine learning algorithms: Decision Trees, KNN and Naive Bayes. Overall, the algorithms resulted in accuracy scores higher than 80%, being KNN the method with the highest score (around 87%). Some of the reasons that might have influenced negatively in the accuracy score on each algorithm: Decision Tree might have overfitted the input data; a not well balance sample for KNN, which might had biased the status prediction; Naive Bayes: the accuracy score might be influenced by high variance of *maximum_wind* data. 